## Import all the important libraries

In [43]:
import os
import numpy as np
import soundfile as sf
import librosa as lbs
import scipy 
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Normalized the audio signals 

In [48]:
# Normalizing the bus signals
input_folder = "Bus Testing Set"
output_folder = "Normalized Testing Bus Sounds"
os.makedirs(output_folder, exist_ok=True)
for file in os.listdir(input_folder):
    if file.endswith(".wav"):
        path = os.path.join(input_folder, file)
        x, sr = sf.read(path)
        x = x.astype(np.float32)
        # Standardization
        mean = np.mean(x)
        std = np.std(x)
        if std < 1e-8:
            x_norm = x - mean
        else:
            x_norm = (x - mean) / std
        # Save normalized file
        out_path = os.path.join(output_folder, file)
        sf.write(out_path, x_norm, sr)

# Normalizing the tram sounds
input_folder = "Tram Testing Set"
output_folder = "Normalized Testing Tram Sounds"
os.makedirs(output_folder, exist_ok=True)
for file in os.listdir(input_folder):
    if file.endswith(".wav"):
        path = os.path.join(input_folder, file)
        x, sr = sf.read(path)
        x = x.astype(np.float32)
        # Standardization
        mean = np.mean(x)
        std = np.std(x)
        if std < 1e-8:
            x_norm = x - mean
        else:
            x_norm = (x - mean) / std
        # Save normalized file
        out_path = os.path.join(output_folder, file)
        sf.write(out_path, x_norm, sr)

## Feature extraction from normalized audio

Run this after normalization. It reads `Normalized Bus Sounds` `Normalized Tram Sounds
`, computes energy, RMS, power spectrogram, mel spectrogram, MFCC, and CQT

In [49]:
# Extract features from normalized audio

input_root = "Normalized Dataset Testing"
classes = {"bus": 0, "tram": 1}


feature_root = "Extracted_Features_Testing"
os.makedirs(feature_root, exist_ok=True)


n_fft = 2048
hop = 512
n_mels = 40
n_mfcc = 24

for label_name, label_id in classes.items():
    input_folder = os.path.join(input_root, label_name)
    output_folder = os.path.join(feature_root, label_name)
    os.makedirs(output_folder, exist_ok=True)
    
    for file in sorted(os.listdir(input_folder)):
        if not file.endswith(".wav"):
            continue

        path = os.path.join(input_folder, file)

        y, sr = lbs.load(path, sr=None, mono=True)

        # Feature extraction
        energy = np.sum(y ** 2)

        rms = lbs.feature.rms(y=y, frame_length=n_fft, hop_length=hop).squeeze()

        power_spec = np.abs(lbs.stft(y, n_fft=n_fft, hop_length=hop)) ** 2

        mel_spec = lbs.feature.melspectrogram(
            S=power_spec, sr=sr, n_mels=n_mels
        )

        mfcc = lbs.feature.mfcc(
            S=lbs.power_to_db(mel_spec), sr=sr, n_mfcc=n_mfcc
        )

        cqt = np.abs(lbs.cqt(
            y=y, sr=sr, hop_length=hop, n_bins=84, bins_per_octave=12
        ))

        # Save into file for ML model
        base = file.replace(".wav", "")

        np.save(os.path.join(output_folder, f"{base}_energy.npy"), energy)
        np.save(os.path.join(output_folder, f"{base}_rms.npy"), rms)
        np.save(os.path.join(output_folder, f"{base}_mel.npy"), mel_spec)
        np.save(os.path.join(output_folder, f"{base}_mfcc.npy"), mfcc)
        np.save(os.path.join(output_folder, f"{base}_cqt.npy"), cqt)


## Defining ML Model for classifying the Audio